In [15]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
train_dataset = datasets.MNIST(root='Data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='Data', train=False, transform=transforms.ToTensor(), download=True)

In [18]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

### नमूना

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=(1,1))   #28/2
        self.conv2 = nn.Conv2d(8, 16, 3, padding=(1,1))    #14/2
        self.pool = nn.MaxPool2d((2,2))   # kernel_size
        self.fc1 = nn.Linear(16*7*7, 10) 
    def forward(self, x):
        x = self.conv1(x)     
        x = F.relu(x)       # Applying relu on the features passed through hidden layer 1
        x = self.pool(x)    # Applying pooling
        x = self.conv2(x)     # passing features throgh hidden layer 2
        x = F.relu(x)
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = F.softmax(x)
        return x
    
model = CNN().to(device)
print(model)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=784, out_features=10, bias=True)
)


In [27]:
def save_checkpoints(state, filename='checkpoint.pth.tar'):torch.save(state, filename)

def load_checkpoints(checkpoints):
    model.load_state_dict(checkpoints['state_dict'])
    optimizer.load_state_dict(checkpoints['optimizer'])



### प्रशिक्षण

In [23]:
num_epochs = 1
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

torch.load('checkpoints.pth.tar')

for epoch in range(num_epochs):
    for batch_data in train_loader:
        
        checkpoints = {'state_dict':model.state_dict(), 'optimizer':optimizer.state_dict()}
        torch.save(checkpoints, 'checkpoint.pth.tar')
        
        
        X, y = batch_data    # X(batch of features) & y(batch of corresponding labels)
        optimizer.zero_grad()
        X = X.to(device)
        y = y.to(device)    
        
        y_ = model(X)
        loss = loss_function(y_, y)
        loss.backward()
        optimizer.step()

<ipython-input-20-e4ee6cb1eb16>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


### परीक्षण

In [25]:
correct = 0
total = 0

for batch_data in test_loader:
    X, y = batch_data
    X = X.to(device)
    y = y.to(device)
    y_ = model(X)
    for idx, i in enumerate(y_):
        if torch.argmax(i) == y[idx]:correct += 1
        total += 1

<ipython-input-20-e4ee6cb1eb16>:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


### यथार्थता

In [26]:
correct/total

0.8644166666666667